In [1]:
# Cell 1: Install Packages
!pip install -q -U nemo_toolkit['asr'] pydub pandas fastapi uvicorn python-multipart pyngrok torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 wget
!pip install numpy==1.26.4 --force-reinstall
!pip install moviepy
!pip install soundfile

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 11.4 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 2: Imports and Global Model Loading
import nemo.collections.asr as nemo_asr
import torch
from pathlib import Path
from pydub import AudioSegment
import os
import shutil
import uvicorn
import nest_asyncio
import asyncio
from pyngrok import ngrok, conf
import subprocess
import tempfile
from moviepy.editor import VideoFileClip
import soundfile as sf
import math

from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse

# --- Global ASR Model Setup ---
print("Loading ASR model globally...")
try:
    asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v2")
    asr_model.eval()
    if torch.cuda.is_available():
        print("Moving model to GPU and using half precision.")
        asr_model.cuda()
        asr_model.half() 
    else:
        print("CUDA not available, model will run on CPU.")
    print("ASR Model loaded successfully.")
except Exception as e:
    print(f"Error loading ASR model: {e}")
    asr_model = None

EXAMPLE_AUDIO_FILENAME = "example_audio_nemo.wav"
EXAMPLE_AUDIO_DOWNLOAD_URL = "https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav"
KAGGLE_WORKING_DIR = Path("/kaggle/working/")
EXAMPLE_AUDIO_PATH = KAGGLE_WORKING_DIR / EXAMPLE_AUDIO_FILENAME

TEMP_UPLOAD_DIR_API = KAGGLE_WORKING_DIR / "api_temp_uploads"
TEMP_UPLOAD_DIR_API.mkdir(parents=True, exist_ok=True)

PROCESSED_AUDIO_DIR_API = KAGGLE_WORKING_DIR / "api_processed_audio"
PROCESSED_AUDIO_DIR_API.mkdir(parents=True, exist_ok=True)

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

Loading ASR model globally...


parakeet-tdt-0.6b-v2.nemo:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

[NeMo I 2025-05-15 18:31:46 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-05-15 18:31:47 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-05-15 18:31:47 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config :

[NeMo I 2025-05-15 18:31:47 nemo_logging:393] PADDING: 0
[NeMo I 2025-05-15 18:31:54 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-15 18:31:54 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-15 18:31:54 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-15 18:31:57 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v2/snapshots/50aec6a056e85b9f95b612df08a2bddc55b58714/parakeet-tdt-0.6b-v2.nemo.
Moving model to GPU and using half precision.
ASR Model loaded successfully.


In [3]:
# Cell 3: Download Example Audio File
if asr_model is not None:
    print(f"Checking for example audio: {EXAMPLE_AUDIO_PATH}")
    if not EXAMPLE_AUDIO_PATH.exists() or EXAMPLE_AUDIO_PATH.stat().st_size == 0:
        print(f"Downloading example audio to {EXAMPLE_AUDIO_PATH}...")
        try:
            subprocess.run(["wget", "-O", str(EXAMPLE_AUDIO_PATH), EXAMPLE_AUDIO_DOWNLOAD_URL], check=True, capture_output=True, text=True)
            print("Download complete.")
            if EXAMPLE_AUDIO_PATH.exists() and EXAMPLE_AUDIO_PATH.stat().st_size > 0:
                print(f"Example audio file {EXAMPLE_AUDIO_PATH} is present and not empty.")
            else:
                print(f"Error: Example audio file {EXAMPLE_AUDIO_PATH} was not downloaded correctly or is empty after wget attempt.")
        except subprocess.CalledProcessError as e:
            print(f"Error downloading example audio with wget: {e}")
            print(f"stderr: {e.stderr}")
        except FileNotFoundError:
            print("Error: wget command not found. Please ensure wget is installed in the Kaggle environment.")
        except Exception as e_download:
            print(f"An unexpected error occurred during download: {e_download}")
    else:
        print(f"Example audio file {EXAMPLE_AUDIO_PATH} already exists.")
else:
    print("ASR model did not load, skipping example audio download.")

Checking for example audio: /kaggle/working/example_audio_nemo.wav
Download complete.
Example audio file /kaggle/working/example_audio_nemo.wav is present and not empty.


In [7]:
# Cell 4: Audio Preprocessing, Extraction, Chunking Functions, and Timecode Formatting

def preprocess_audio_for_nemo(input_audio_path_str: str, output_processing_dir: Path) -> str:
    """
    Preprocesses an audio file to be 16kHz mono WAV for NeMo.
    Saves the processed file in output_processing_dir.
    Returns the path (as a string) to the processed file.
    """
    print(f"Starting preprocessing for: {input_audio_path_str}")
    audio_path_obj = Path(input_audio_path_str)
    if not audio_path_obj.exists():
        print(f"Error: Input audio file not found at {input_audio_path_str}")
        raise FileNotFoundError(f"Input audio file not found: {input_audio_path_str}")

    try:
        audio = AudioSegment.from_file(input_audio_path_str)
        print(f"Original audio - Channels: {audio.channels}, Frame Rate: {audio.frame_rate}, Duration: {audio.duration_seconds:.2f}s")
    except Exception as e:
        print(f"Error loading audio file {input_audio_path_str} with pydub: {e}")
        raise

    resampled = False
    mono = False
    needs_export = False

    target_sr = 16000
    if audio.frame_rate != target_sr:
        print(f"Resampling audio from {audio.frame_rate}Hz to {target_sr}Hz")
        audio = audio.set_frame_rate(target_sr)
        resampled = True
        needs_export = True

    if audio.channels != 1:
        print(f"Converting to mono (from {audio.channels} channels)")
        audio = audio.set_channels(1)
        mono = True
        needs_export = True

    processed_filename = f"{audio_path_obj.stem}_processed_16kHz_mono.wav"
    processed_path_str = str(output_processing_dir / processed_filename)

    if needs_export:
        try:
            audio.export(processed_path_str, format="wav")
            print(f"Processed audio saved to: {processed_path_str}")
        except Exception as e:
            print(f"Error exporting processed audio to {processed_path_str}: {e}")
            raise
        return processed_path_str
    else:
        if str(audio_path_obj.resolve()) != str(Path(processed_path_str).resolve()):
            print(f"Audio was already 16kHz mono. Copying to target processing directory: {processed_path_str}")
            shutil.copy(input_audio_path_str, processed_path_str)
            return processed_path_str
        else:
            print("Audio already 16kHz mono and in the correct location/name.")
            return input_audio_path_str

def extract_audio_from_video(video_path: str, output_dir: Path) -> str:
    """
    Extracts audio from a video file and saves it as WAV.
    Returns the path to the extracted audio file.
    """
    video_path_obj = Path(video_path)
    output_audio_path = str(output_dir / f"{video_path_obj.stem}_audio.wav")
    try:
        print(f"Extracting audio from video: {video_path}")
        video = VideoFileClip(video_path)
        audio = video.audio
        if audio is None:
            raise ValueError("No audio stream found in the video file")
        print(f"Saving extracted audio to: {output_audio_path}")
        audio.write_audiofile(output_audio_path, fps=16000, nbytes=2, codec='pcm_s16le')
        video.close()
        return output_audio_path
    except Exception as e:
        print(f"Error extracting audio from video: {e}")
        raise

def split_audio_into_chunks(audio_path: str, chunk_length_sec: int = 60, output_dir: Path = None) -> list:
    """
    Splits a WAV audio file into chunks of chunk_length_sec seconds.
    Returns a list of chunk file paths.
    """
    print(f"Splitting audio into {chunk_length_sec}s chunks: {audio_path}")
    import soundfile as sf
    import math
    y, sr = sf.read(audio_path)
    total_samples = len(y)
    samples_per_chunk = chunk_length_sec * sr
    num_chunks = math.ceil(total_samples / samples_per_chunk)
    chunk_paths = []
    audio_path_obj = Path(audio_path)
    if output_dir is None:
        output_dir = audio_path_obj.parent
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    for i in range(num_chunks):
        start = int(i * samples_per_chunk)
        end = int(min((i + 1) * samples_per_chunk, total_samples))
        chunk_data = y[start:end]
        chunk_path = output_dir / f"{audio_path_obj.stem}_chunk_{i+1:03d}.wav"
        sf.write(chunk_path, chunk_data, sr)
        chunk_paths.append(str(chunk_path))
        print(f"Chunk {i+1}: {chunk_path} ({(end-start)/sr:.2f}s)")
    return chunk_paths

def seconds_to_mmss(seconds: float) -> str:
    """Convierte segundos a formato mm:ss"""
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m:02d}:{s:02d}"

In [8]:
# Cell 5: FastAPI Application with Chunked Video Processing Endpoint (mm:ss timecodes)
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*", "http://localhost:3000", "https://localhost:3000", "http://localhost", "http://127.0.0.1:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)

@app.post("/transcribe_video_chunked/")
async def transcribe_video_chunked_endpoint(uploaded_file: UploadFile = File(...)):
    if asr_model is None:
        print("Error: ASR model is not available (was not loaded in Cell 2).")
        raise HTTPException(status_code=503, detail="Transcription service is currently unavailable (model not loaded).")

    if not uploaded_file or not uploaded_file.filename:
        raise HTTPException(status_code=400, detail="No file uploaded")
    
    video_path = None
    audio_path = None
    processed_audio_path = None
    chunk_paths = []
    try:
        # Save uploaded video to temp file
        with tempfile.NamedTemporaryFile(delete=False, suffix=Path(uploaded_file.filename).suffix) as tmp:
            video_path = tmp.name
            content = await uploaded_file.read()
            tmp.write(content)
        print(f"Video saved temporarily to: {video_path}")

        # Extract audio from video
        audio_path = extract_audio_from_video(video_path, TEMP_UPLOAD_DIR_API)

        # Preprocess audio (16kHz mono)
        processed_audio_path = preprocess_audio_for_nemo(audio_path, PROCESSED_AUDIO_DIR_API)

        # Split audio into 1-minute chunks
        chunk_paths = split_audio_into_chunks(processed_audio_path, chunk_length_sec=60, output_dir=PROCESSED_AUDIO_DIR_API)

        # Transcribe each chunk and concatenate results
        all_transcripts = []
        for idx, chunk_path in enumerate(chunk_paths):
            print(f"Transcribing chunk {idx+1}/{len(chunk_paths)}: {chunk_path}")
            if torch.cuda.is_available():
                if not next(asr_model.parameters()).is_cuda:
                    asr_model.cuda()
                if next(asr_model.parameters()).data.dtype != torch.float16:
                    if hasattr(asr_model, 'half') and callable(asr_model.half):
                        asr_model.half()
            hypotheses = asr_model.transcribe([chunk_path], timestamps=True)
            if not hypotheses or not isinstance(hypotheses, list) or not hypotheses[0]:
                print(f"Warning: No transcription for chunk {idx+1}")
                continue
            first_hypothesis = hypotheses[0]
            if not hasattr(first_hypothesis, 'timestamp') or 'segment' not in first_hypothesis.timestamp:
                print(f"Unexpected transcription output format in chunk {idx+1}")
                continue
            segment_data = first_hypothesis.timestamp['segment']
            for s_data_dict in segment_data:
                if not isinstance(s_data_dict, dict) or not all(k in s_data_dict for k in ['start', 'end', 'segment']):
                    continue
                start_time = s_data_dict['start'] + idx * 60
                end_time = s_data_dict['end'] + idx * 60
                text_segment = s_data_dict['segment']
                # Timecodes en formato mm:ss
                all_transcripts.append(f"{seconds_to_mmss(start_time)} - {seconds_to_mmss(end_time)} : {text_segment}")

        final_transcript_text = "\n".join(all_transcripts)
        print("Full video transcription (chunked) complete")
        return JSONResponse(content={
            "filename": f"{Path(uploaded_file.filename).stem}_transcription.txt",
            "transcription_text": final_transcript_text
        })
    except Exception as e:
        print(f"Error during chunked video transcription: {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"An error occurred during transcription: {str(e)}")
    finally:
        # Clean up temporary files
        for path in [video_path, audio_path, processed_audio_path] + chunk_paths:
            if path and Path(path).exists():
                try:
                    Path(path).unlink()
                    print(f"Removed temporary file: {path}")
                except OSError as e:
                    print(f"Error removing temporary file: {path} - {e}")

In [10]:
# Cell 6: Start Ngrok and Uvicorn Server
NGROK_AUTH_TOKEN = "2x43w3eV98ow7s4Esh2761K7Xib_5piK9Ef22bp5q45ZhT9E3" # <--- REEMPLAZA POR TU TOKEN

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTHTOKEN" or not NGROK_AUTH_TOKEN or NGROK_AUTH_TOKEN == "TU_TOKEN_NGROK_AQUI":
    print("CRITICAL: Please set your NGROK_AUTH_TOKEN in Cell 6.")
elif asr_model is None:
    print("CRITICAL: ASR model failed to load (see Cell 2). API will not start.")
elif not EXAMPLE_AUDIO_PATH.exists() or EXAMPLE_AUDIO_PATH.stat().st_size == 0:
    print(f"CRITICAL: Example audio file {EXAMPLE_AUDIO_PATH} is missing or empty (see Cell 3). API will not start.")
else:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    try:
        tunnels = ngrok.get_tunnels()
        for tunnel in tunnels:
            ngrok.disconnect(tunnel.public_url)
            print(f"Disconnected previous tunnel: {tunnel.public_url}")
        ngrok.kill()
        print("Killed all ngrok processes.")
    except Exception as e_ngrok_ops:
        print(f"Error managing ngrok processes (this is often okay if none were running): {e_ngrok_ops}")

    PORT = 7860 
    try:
        public_url_obj = ngrok.connect(PORT)
        if hasattr(public_url_obj, 'public_url'):
            public_url_str = public_url_obj.public_url
        else:
            public_url_str = str(public_url_obj).replace('NgrokTunnel: \"', '').replace('\" -> \"http://localhost:7860\"', '')
        if 'ngrok-free.app' in public_url_str and not public_url_str.startswith('http'):
            public_url_str = f"https://{public_url_str}"

        print(f"Ngrok tunnel active: {public_url_str}")
        video_endpoint = f"{public_url_str}/transcribe_video_chunked/"
        print(f"Your chunked video transcription API endpoint will be: {video_endpoint}")
        print("\n*** IMPORTANT: UPDATE YOUR FRONTEND API.TS WITH THIS ENDPOINT ***")
        print(f"const response = await fetch('{video_endpoint}', {{")

        print(f"\nStarting Uvicorn server on port {PORT}...")
        import uvicorn.config
        import asyncio
        import threading

        server_config = uvicorn.config.Config(app=app, host="0.0.0.0", port=PORT, log_level="info")
        server = uvicorn.server.Server(config=server_config)

        def run_server():
            asyncio.set_event_loop(asyncio.new_event_loop())
            server.run()

        thread = threading.Thread(target=run_server, daemon=True)
        thread.start()

        import time
        import requests
        print("Waiting for server to start...")
        time.sleep(2)
        try:
            health_check = requests.get(f"http://localhost:{PORT}")
            print(f"Server health check: {health_check.status_code}")
            print(f"Server is running in background thread. Ngrok tunnel is active.")
            print("The API will continue running until you stop the notebook or kernel.")
            print("\n*** After the server is running, test your endpoint with this command: ***")
            print(f"curl -X POST {video_endpoint} -F \"uploaded_file=@/path/to/your/video.mp4\"")
        except Exception as e:
            print(f"Server may not be fully started yet: {e}")
            print("Continue with the next steps anyway, the server should start soon.")
    except Exception as e_ngrok_run:
        print(f"Could not start ngrok or Uvicorn server: {e_ngrok_run}")
        import traceback
        traceback.print_exc()

Killed all ngrok processes.
Ngrok tunnel active: https://105b-34-139-97-175.ngrok-free.app
Your chunked video transcription API endpoint will be: https://105b-34-139-97-175.ngrok-free.app/transcribe_video_chunked/

*** IMPORTANT: UPDATE YOUR FRONTEND API.TS WITH THIS ENDPOINT ***
const response = await fetch('https://105b-34-139-97-175.ngrok-free.app/transcribe_video_chunked/', {

Starting Uvicorn server on port 7860...


INFO:     Started server process [35]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 7860): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Waiting for server to start...
INFO:     127.0.0.1:52322 - "GET / HTTP/1.1" 404 Not Found
Server health check: 404
Server is running in background thread. Ngrok tunnel is active.
The API will continue running until you stop the notebook or kernel.

*** After the server is running, test your endpoint with this command: ***
curl -X POST https://105b-34-139-97-175.ngrok-free.app/transcribe_video_chunked/ -F "uploaded_file=@/path/to/your/video.mp4"
Video saved temporarily to: /tmp/tmp0b100zdw.mp4
Extracting audio from video: /tmp/tmp0b100zdw.mp4
Saving extracted audio to: /kaggle/working/api_temp_uploads/tmp0b100zdw_audio.wav
MoviePy - Writing audio in /kaggle/working/api_temp_uploads/tmp0b100zdw_audio.wav


MoviePy - Done.
Starting preprocessing for: /kaggle/working/api_temp_uploads/tmp0b100zdw_audio.wav
Original audio - Channels: 2, Frame Rate: 16000, Duration: 1010.94s
Converting to mono (from 2 channels)
Processed audio saved to: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono.wav
Splitting audio into 60s chunks: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono.wav
Chunk 1: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_001.wav (60.00s)
Chunk 2: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_002.wav (60.00s)
Chunk 3: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_003.wav (60.00s)
Chunk 4: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_004.wav (60.00s)
Chunk 5: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_005.wav (60.00s)
Chunk 6: /kaggle/working/api_processed_audio

[NeMo W 2025-05-15 18:45:44 nemo_logging:405] `include_duration` is not implemented for CUDA graphs
Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]

...

Transcribing chunk 16/17: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_016.wav
[NeMo I 2025-05-15 18:45:56 nemo_logging:393] Timestamps requested, setting decoding timestamps to True. Capture them in Hypothesis object,                         with output[0][idx].timestep['word'/'segment'/'char']
[NeMo I 2025-05-15 18:45:56 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}



[NeMo W 2025-05-15 18:45:56 nemo_logging:405] `include_duration` is not implemented for CUDA graphs
Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]

Transcribing chunk 17/17: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_017.wav
[NeMo I 2025-05-15 18:45:57 nemo_logging:393] Timestamps requested, setting decoding timestamps to True. Capture them in Hypothesis object,                         with output[0][idx].timestep['word'/'segment'/'char']
[NeMo I 2025-05-15 18:45:57 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}



[NeMo W 2025-05-15 18:45:57 nemo_logging:405] `include_duration` is not implemented for CUDA graphs
Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]

Full video transcription (chunked) complete
Removed temporary file: /tmp/tmp0b100zdw.mp4
Removed temporary file: /kaggle/working/api_temp_uploads/tmp0b100zdw_audio.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_001.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_002.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_003.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_004.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_005.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmp0b100zdw_audio_processed_16kHz_mono_chunk_006.wav
Removed temporary file: /kaggle/working/api_process